In [ ]:
pip install rake-nltk

In [1]:
import pandas as pd
from rake_nltk import Rake
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

pd.set_option('display.max_columns', 100)
df = pd.read_csv('movie_metadata.csv')
df.head()

,color,director_name,num_critic_for_reviews,duration,director_facebook_likes,actor_3_facebook_likes,actor_2_name,actor_1_facebook_likes,gross,genres,actor_1_name,movie_title,num_voted_users,cast_total_facebook_likes,actor_3_name,facenumber_in_poster,plot_keywords,movie_imdb_link,num_user_for_reviews,language,country,content_rating,budget,title_year,actor_2_facebook_likes,imdb_score,aspect_ratio,movie_facebook_likes
0,Color,James Cameron,723.0,178.0,0.0,855.0,Joel David Moore,1000.0,760505847.0,Action|Adventure|Fantasy|Sci-Fi,CCH Pounder,Avatar,886204,4834,Wes Studi,0.0,avatar|future|marine|native|paraplegic,http://www.imdb.com/title/tt0499549/?ref_=fn_t...,3054.0,English,USA,PG-13,237000000.0,2009.0,936.0,7.9,1.78,33000
1,Color,Gore Verbinski,302.0,169.0,563.0,1000.0,Orlando Bloom,40000.0,309404152.0,Action|Adventure|Fantasy,Johnny Depp,Pirates of the Caribbean: At World's End,471220,48350,Jack Davenport,0.0,goddess|marriage ceremony|marriage proposal|pi...,http://www.imdb.com/title/tt0449088/?ref_=fn_t...,1238.0,English,USA,PG-13,300000000.0,2007.0,5000.0,7.1,2.35,0
2,Color,Sam Mendes,602.0,148.0,0.0,161.0,Rory Kinnear,11000.0,200074175.0,Action|Adventure|Thriller,Christoph Waltz,Spectre,275868,11700,Stephanie Sigman,1.0,bomb|espionage|sequel|spy|terrorist,http://www.imdb.com/title/tt2379713/?ref_=fn_t...,994.0,English,UK,PG-13,245000000.0,2015.0,393.0,6.8,2.35,85000
3,Color,Christopher Nolan,813.0,164.0,22000.0,23000.0,Christian Bale,27000.0,448130642.0,Action|Thriller,Tom Hardy,The Dark Knight Rises,1144337,106759,Joseph Gordon-Levitt,0.0,deception|imprisonment|lawlessness|police offi...,http://www.imdb.com/title/tt1345836/?ref_=fn_t...,2701.0,English,USA,PG-13,250000000.0,2012.0,23000.0,8.5,2.35,164000
4,NaN,Doug Walker,NaN,NaN,131.0,NaN,Rob Walker,131.0,NaN,Documentary,Doug Walker,Star Wars: Episode VII - The Force Awakens ...,8,143,NaN,0.0,NaN,http://www.imdb.com/title/tt5289954/?ref_=fn_t...,NaN,NaN,NaN,NaN,NaN,NaN,12.0,7.1,NaN,0


In [2]:
df.shape
list(df.columns.values)

['color',
 'director_name',
 'num_critic_for_reviews',
 'duration',
 'director_facebook_likes',
 'actor_3_facebook_likes',
 'actor_2_name',
 'actor_1_facebook_likes',
 'gross',
 'genres',
 'actor_1_name',
 'movie_title',
 'num_voted_users',
 'cast_total_facebook_likes',
 'actor_3_name',
 'facenumber_in_poster',
 'plot_keywords',
 'movie_imdb_link',
 'num_user_for_reviews',
 'language',
 'country',
 'content_rating',
 'budget',
 'title_year',
 'actor_2_facebook_likes',
 'imdb_score',
 'aspect_ratio',
 'movie_facebook_likes']

In [3]:
df = df[['director_name', 'actor_1_name', 'actor_2_name', 'actor_3_name', 'plot_keywords', 'genres', 'movie_title']]

In [4]:
if not df['actor_1_name'].empty or not df['actor_2_name'].empty or not df['actor_3_name'].empty:
    df['actors'] = df['actor_1_name'] + "," +  df['actor_2_name'] + "," +  df['actor_3_name'] 

In [5]:
df = df[['director_name', 'plot_keywords', 'genres', 'movie_title', 'actors']]
df.dropna()
df.head()

,director_name,plot_keywords,genres,movie_title,actors
0,James Cameron,avatar|future|marine|native|paraplegic,Action|Adventure|Fantasy|Sci-Fi,Avatar,"CCH Pounder,Joel David Moore,Wes Studi"
1,Gore Verbinski,goddess|marriage ceremony|marriage proposal|pi...,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,"Johnny Depp,Orlando Bloom,Jack Davenport"
2,Sam Mendes,bomb|espionage|sequel|spy|terrorist,Action|Adventure|Thriller,Spectre,"Christoph Waltz,Rory Kinnear,Stephanie Sigman"
3,Christopher Nolan,deception|imprisonment|lawlessness|police offi...,Action|Thriller,The Dark Knight Rises,"Tom Hardy,Christian Bale,Joseph Gordon-Levitt"
4,Doug Walker,NaN,Documentary,Star Wars: Episode VII - The Force Awakens ...,NaN


In [6]:
df1 = df.where((pd.notnull(df)), 'REMOVE')
df1.head()

,director_name,plot_keywords,genres,movie_title,actors
0,James Cameron,avatar|future|marine|native|paraplegic,Action|Adventure|Fantasy|Sci-Fi,Avatar,"CCH Pounder,Joel David Moore,Wes Studi"
1,Gore Verbinski,goddess|marriage ceremony|marriage proposal|pi...,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,"Johnny Depp,Orlando Bloom,Jack Davenport"
2,Sam Mendes,bomb|espionage|sequel|spy|terrorist,Action|Adventure|Thriller,Spectre,"Christoph Waltz,Rory Kinnear,Stephanie Sigman"
3,Christopher Nolan,deception|imprisonment|lawlessness|police offi...,Action|Thriller,The Dark Knight Rises,"Tom Hardy,Christian Bale,Joseph Gordon-Levitt"
4,Doug Walker,REMOVE,Documentary,Star Wars: Episode VII - The Force Awakens ...,REMOVE


In [7]:
df.replace(["NaN"], np.nan, inplace = True)
df = df.dropna()
df.head()

,director_name,plot_keywords,genres,movie_title,actors
0,James Cameron,avatar|future|marine|native|paraplegic,Action|Adventure|Fantasy|Sci-Fi,Avatar,"CCH Pounder,Joel David Moore,Wes Studi"
1,Gore Verbinski,goddess|marriage ceremony|marriage proposal|pi...,Action|Adventure|Fantasy,Pirates of the Caribbean: At World's End,"Johnny Depp,Orlando Bloom,Jack Davenport"
2,Sam Mendes,bomb|espionage|sequel|spy|terrorist,Action|Adventure|Thriller,Spectre,"Christoph Waltz,Rory Kinnear,Stephanie Sigman"
3,Christopher Nolan,deception|imprisonment|lawlessness|police offi...,Action|Thriller,The Dark Knight Rises,"Tom Hardy,Christian Bale,Joseph Gordon-Levitt"
5,Andrew Stanton,alien|american civil war|male nipple|mars|prin...,Action|Adventure|Sci-Fi,John Carter,"Daryl Sabara,Samantha Morton,Polly Walker"


In [8]:
for index, row in df.iterrows():
    # process actors names
    app = row['actors'].lower().replace(' ','')
    app = app.replace(',', ' ')
    row['actors'] = app
    
    # process director_name
    app = row['director_name'].lower().replace(' ', '')
    row['director_name'] = app
    
    # process genres
    app = row['genres'].lower().replace('|', ' ')
    row['genres'] = app
    
    # process plot_keywords
    app = row['plot_keywords'].lower().replace('|', ' ')
    row['plot_keywords'] = app
    
    
df.head()

,director_name,plot_keywords,genres,movie_title,actors
0,jamescameron,avatar future marine native paraplegic,action adventure fantasy sci-fi,Avatar,cchpounder joeldavidmoore wesstudi
1,goreverbinski,goddess marriage ceremony marriage proposal pi...,action adventure fantasy,Pirates of the Caribbean: At World's End,johnnydepp orlandobloom jackdavenport
2,sammendes,bomb espionage sequel spy terrorist,action adventure thriller,Spectre,christophwaltz rorykinnear stephaniesigman
3,christophernolan,deception imprisonment lawlessness police offi...,action thriller,The Dark Knight Rises,tomhardy christianbale josephgordon-levitt
5,andrewstanton,alien american civil war male nipple mars prin...,action adventure sci-fi,John Carter,darylsabara samanthamorton pollywalker


In [9]:
df.set_index('movie_title', inplace = True)
df.head()

,director_name,plot_keywords,genres,actors
movie_title,,,,
Avatar,jamescameron,avatar future marine native paraplegic,action adventure fantasy sci-fi,cchpounder joeldavidmoore wesstudi
Pirates of the Caribbean: At World's End,goreverbinski,goddess marriage ceremony marriage proposal pi...,action adventure fantasy,johnnydepp orlandobloom jackdavenport
Spectre,sammendes,bomb espionage sequel spy terrorist,action adventure thriller,christophwaltz rorykinnear stephaniesigman
The Dark Knight Rises,christophernolan,deception imprisonment lawlessness police offi...,action thriller,tomhardy christianbale josephgordon-levitt
John Carter,andrewstanton,alien american civil war male nipple mars prin...,action adventure sci-fi,darylsabara samanthamorton pollywalker


In [10]:
df['bag_of_words'] = ''
columns = df.columns
for index, row in df.iterrows():
    words = ''
    for col in columns:
        words = words + row[col]+ ' '
    row['bag_of_words'] = words
    
df.drop(columns = [col for col in df.columns if col!= 'bag_of_words'], inplace = True)

In [11]:
df.head()

,bag_of_words
movie_title,
Avatar,jamescameron avatar future marine native parap...
Pirates of the Caribbean: At World's End,goreverbinski goddess marriage ceremony marria...
Spectre,sammendes bomb espionage sequel spy terrorist ...
The Dark Knight Rises,christophernolan deception imprisonment lawles...
John Carter,andrewstanton alien american civil war male ni...


In [12]:
# instantiating and generating the count matrix
count = CountVectorizer()
count_matrix = count.fit_transform(df['bag_of_words'])

# creating a Series for the movie titles so they are associated to an ordered numerical
# list I will use later to match the indexes
indices = pd.Series(df.index)
indices[:5]

0                                      Avatar 
1    Pirates of the Caribbean: At World's End 
2                                     Spectre 
3                       The Dark Knight Rises 
4                                 John Carter 
Name: movie_title, dtype: object

In [13]:
# generating the cosine similarity matrix
cosine_sim = cosine_similarity(count_matrix, count_matrix)
cosine_sim

array([[1.        , 0.20044593, 0.15430335, ..., 0.        , 0.        ,
        0.        ],
       [0.20044593, 1.        , 0.14433757, ..., 0.        , 0.        ,
        0.        ],
       [0.15430335, 0.14433757, 1.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 1.        , 0.16666667,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.16666667, 1.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        1.        ]])

In [33]:
# function that takes in movie title as input and returns the top 10 recommended movies
def recommendations(title, cosine_sim = cosine_sim):
    
    recommended_movies = []
    idx = -1
    # gettin the index of the movie that matches the title
    for i in range(0, indices.size):
        if indices[i] == title:
            idx = i
            break

    # creating a Series with the similarity scores in descending order
    score_series = pd.Series(cosine_sim[idx]).sort_values(ascending = False)

    # getting the indexes of the 10 most similar movies
    top_10_indexes = list(score_series.iloc[1:11].index)
    
    # populating the list with the titles of the best 10 matching movies
    for i in top_10_indexes:
        recommended_movies.append(list(df.index)[i])
        
    return recommended_movies

In [35]:
recommendations('Batman v Superman: Dawn of Justice ')

['Man of Steel\xa0',
 'Suicide Squad\xa0',
 'Avengers: Age of Ultron\xa0',
 'Superman II\xa0',
 'Captain America: Civil War\xa0',
 'Teenage Mutant Ninja Turtles: Out of the Shadows\xa0',
 'Spawn\xa0',
 'Deadpool\xa0',
 'The 5th Wave\xa0',
 'Jumper\xa0']